In [1]:
import os

In [2]:
%pwd

'c:\\Users\\91749\\Downloads\\Kidney_Disease_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91749\\Downloads\\Kidney_Disease_Detection'

In [16]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [17]:
import os

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)  # Make sure this returns a dictionary
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config['artifacts_root'])])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']  # Assuming this is a dictionary

        create_directories([Path(config['root_dir'])])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config['root_dir']),
            source_url=config['source_url'],  # Adjusted to match the DataIngestionConfig
            local_data_file=Path(config['local_data_file']),
            unzip_dir=Path(config['unzip_dir'])
        )

        return data_ingestion_config

In [19]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [21]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the URL
        """
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs(zip_download_dir.parent, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, str(zip_download_dir))  # Ensure path is a string

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            logger.error(f"Failed to download file: {e}")
            raise

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"Extracted zip file to {unzip_path}")

        except Exception as e:
            logger.error(f"Failed to extract zip file: {e}")
            raise

In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-03 16:34:02,834: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-03 16:34:02,839: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-03 16:34:02,841: INFO: common: created directory at: artifacts]
[2024-08-03 16:34:02,843: INFO: common: created directory at: artifacts\data_ingestion]
[2024-08-03 16:34:02,847: INFO: 1696747317: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts\data_ingestion\data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=ea8d1825-60d4-401a-a15b-d73231d4b684
To: c:\Users\91749\Downloads\Kidney_Disease_Detection\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:06<00:00, 9.46MB/s]

[2024-08-03 16:34:14,744: INFO: 1696747317: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts\data_ingestion\data.zip]


[2024-08-03 16:34:15,895: INFO: 1696747317: Extracted zip file to artifacts\data_ingestion]
